# Clasificador de Residuos Local - Versión Limpia

Este notebook está optimizado para ejecutarse localmente en Windows sin conflictos.

**Clases de residuos:**
- cardboard (cartón)
- compost (compost)  
- glass (vidrio)
- metal (metal)
- paper (papel)
- plastic (plástico)
- trash (basura general)

## 📋 Instrucciones de uso:
1. Ejecuta las celdas en orden secuencial
2. La celda 5 copia solo imágenes nuevas (sin duplicar)
3. La celda 6 clasifica automáticamente la primera imagen
4. La celda 7 permite seleccionar cualquier imagen por índice
5. La celda 8 permite clasificar por ruta específica


In [ ]:
# Instalar dependencias (ejecutar solo una vez)
%pip install fastai==2.7.12 opencv-python matplotlib pillow


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Importar librerías
from fastai.vision.all import *
import cv2
import matplotlib.pyplot as plt
import zipfile
import io
import os
from PIL import Image

print("✅ Librerías importadas correctamente")


✅ Librerías importadas correctamente


In [3]:
# Descargar repositorio si no existe
import subprocess

if not os.path.exists("fastai-waste-classifier"):
    print("📁 Descargando repositorio...")
    subprocess.run(["git", "clone", "https://github.com/rootstrap/fastai-waste-classifier"], 
                   check=True, capture_output=True)
    print("✅ Repositorio descargado")
else:
    print("✅ Repositorio ya existe")


✅ Repositorio ya existe


In [4]:
# Cargar modelo con solución para Windows
import torch
import pickle
import sys
from pathlib import Path, WindowsPath

def load_model_windows_safe(model_path):
    """Cargar modelo de forma segura en Windows"""
    try:
        # Método 1: Usar torch.load con pickle personalizado
        print("🔄 Intentando carga con torch.load...")
        
        class WindowsUnpickler(pickle.Unpickler):
            def find_class(self, module, name):
                if module == 'pathlib' and name == 'PosixPath':
                    return WindowsPath
                return super().find_class(module, name)
        
        with open(model_path, 'rb') as f:
            unpickler = WindowsUnpickler(f)
            model_data = unpickler.load()
        
        print("✅ Modelo cargado con éxito usando torch.load")
        return model_data
        
    except Exception as e:
        print(f"⚠️ Error con método 1: {e}")
        
        # Método 2: Usar load_learner con monkey patching
        try:
            print("🔄 Intentando con monkey patching...")
            
            import pathlib
            original_posix = pathlib.PosixPath
            
            class FakePosixPath(WindowsPath):
                def __new__(cls, *args, **kwargs):
                    return WindowsPath(*args, **kwargs)
            
            pathlib.PosixPath = FakePosixPath
            from fastai.vision.all import load_learner
            learn_loaded = load_learner(model_path)
            pathlib.PosixPath = original_posix
            
            print("✅ Modelo cargado con éxito usando monkey patching")
            return learn_loaded
            
        except Exception as e2:
            print(f"❌ Error con método 2: {e2}")
            return None

model_path = "fastai-waste-classifier/result-resnet50.pkl"

if os.path.exists(model_path):
    print("🔄 Cargando modelo...")
    learn_loaded = load_model_windows_safe(model_path)
    
    if learn_loaded:
        print("✅ Modelo cargado exitosamente")
        try:
            print(f"📋 Clases disponibles: {learn_loaded.dls.vocab}")
        except:
            print("📋 Modelo cargado (sin información de clases)")
    else:
        print("❌ No se pudo cargar el modelo")
else:
    print(f"❌ Error: No se encontró el modelo en {model_path}")


🔄 Cargando modelo...
🔄 Intentando carga con torch.load...
⚠️ Error con método 1: A load persistent id instruction was encountered,
but no persistent_load function was specified.
🔄 Intentando con monkey patching...
✅ Modelo cargado con éxito usando monkey patching
✅ Modelo cargado exitosamente
📋 Clases disponibles: ['cardboard', 'compost', 'glass', 'metal', 'paper', 'plastic', 'trash']


In [5]:
# Función para clasificar imágenes
def predict_image(image_path):
    """Clasificar una imagen individual"""
    try:
        # Mostrar imagen
        img = Image.open(image_path)
        plt.figure(figsize=(8, 6))
        plt.imshow(img)
        plt.title(f"Imagen: {os.path.basename(image_path)}")
        plt.axis('off')
        plt.show()
        
        # Predecir
        prediction = learn_loaded.predict(image_path)
        num = prediction[1].numpy().tolist()
        confidence = prediction[2].numpy()[num]
        
        print(f"🏷️ Clasificado como: {prediction[0]}")
        print(f"📊 Confianza: {confidence:.2%}")
        
        return prediction[0], num, confidence
        
    except Exception as e:
        print(f"❌ Error clasificando imagen: {e}")
        return None, None, None

print("✅ Función de clasificación definida")


✅ Función de clasificación definida


In [ ]:
# Verificar y organizar imágenes en carpeta - VERSIÓN FINAL
import shutil
import glob
import os

def organize_images_final():
    """Verificar que las imágenes estén en la carpeta organizada"""
    test_photos_dir = "fastai-waste-classifier/test-photos"
    target_folder = "fotos_para_clasificar"
    
    print(f"📁 Verificando organización de imágenes...")
    
    # Verificar que existe la carpeta destino
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)
        print(f"📁 Carpeta creada: {target_folder}")
    else:
        print(f"✅ Carpeta existe: {target_folder}")
    
    # Contar imágenes en la carpeta organizada
    images_in_folder = []
    if os.path.exists(target_folder):
        for ext in ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.gif']:
            images_in_folder.extend(glob.glob(os.path.join(target_folder, ext)))
    
    print(f"📸 Imágenes en {target_folder}/: {len(images_in_folder)}")
    
    # Si hay pocas imágenes, copiar desde test-photos
    if len(images_in_folder) < 5 and os.path.exists(test_photos_dir):
        print("📋 Copiando imágenes desde test-photos...")
        
        # Obtener imágenes existentes (solo nombres únicos)
        existing_names = set(os.path.basename(img).lower() for img in images_in_folder)
        
        # Obtener imágenes en test-photos
        test_images = []
        for file in os.listdir(test_photos_dir):
            if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
                test_images.append(file)
        
        # Copiar solo las nuevas
        new_copied = 0
        for file in test_images:
            if file.lower() not in existing_names:
                try:
                    source_path = os.path.join(test_photos_dir, file)
                    target_path = os.path.join(target_folder, file)
                    shutil.copy2(source_path, target_path)
                    print(f"✅ Copiada: {file}")
                    new_copied += 1
                except Exception as e:
                    print(f"❌ Error copiando {file}: {e}")
        
        print(f"📊 Nuevas imágenes copiadas: {new_copied}")
    
    # Mostrar imágenes finales
    final_images = []
    seen_names = set()
    
    if os.path.exists(target_folder):
        for ext in ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.gif']:
            for img in glob.glob(os.path.join(target_folder, ext)):
                name = os.path.basename(img).lower()
                if name not in seen_names:
                    final_images.append(img)
                    seen_names.add(name)
        
        print(f"\n📸 Imágenes organizadas en {target_folder}/ ({len(final_images)}) - SIN DUPLICADOS:")
        for i, img in enumerate(sorted(final_images), 1):
            print(f"  {i}. {os.path.basename(img)}")
    else:
        print(f"\n❌ No se encontró la carpeta {target_folder}/")

# Ejecutar función
organize_images_final()


📁 Verificando organización de imágenes...
✅ Carpeta existe: fotos_para_clasificar
📸 Imágenes en fotos_para_clasificar/: 0
📋 Copiando imágenes desde test-photos...
✅ Copiada: basura-envase-plastico.jpeg
✅ Copiada: basura-vaso-plastico.jpeg
✅ Copiada: basura1-metal.jpeg
✅ Copiada: botella plastico azul.jpg
✅ Copiada: botella verde.jpeg
✅ Copiada: botella vidrio.jpg
✅ Copiada: botella-plastico.jpeg
✅ Copiada: botella.png
✅ Copiada: brocoli.jpg
✅ Copiada: camisa.jpg
✅ Copiada: carro viejo.jpg
✅ Copiada: cascara-banana.jpeg
✅ Copiada: cascara-banana2.jpeg
✅ Copiada: cascara-banana3.jpeg
✅ Copiada: cascara.png
✅ Copiada: clavos.png
✅ Copiada: envase-plastico.jpeg
✅ Copiada: envase-plastico2.jpeg
✅ Copiada: higienico.png
✅ Copiada: Imagen de WhatsApp 2025-10-21 a las 08.55.14_be760169.jpg
✅ Copiada: lata abierta.jpg
✅ Copiada: lata cocacola.png
✅ Copiada: lata.jpeg
✅ Copiada: lata2.jpeg
✅ Copiada: manzana-mordida.jpeg
✅ Copiada: muñeca.jpg
✅ Copiada: papaya.png
✅ Copiada: papel-sucio.jpeg
✅ C

In [22]:
# Limpiar imágenes duplicadas de la carpeta organizada
import os
import glob

def clean_duplicate_images_in_folder():
    """Eliminar imágenes duplicadas de la carpeta organizada"""
    target_folder = "fotos_para_clasificar"
    
    if not os.path.exists(target_folder):
        print(f"❌ No se encontró la carpeta {target_folder}/")
        return
    
    print(f"🧹 Limpiando imágenes duplicadas en {target_folder}/...")
    
    # Buscar todas las imágenes en la carpeta
    image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.gif']
    all_images = []
    
    for ext in image_extensions:
        all_images.extend(glob.glob(os.path.join(target_folder, ext)))
        all_images.extend(glob.glob(os.path.join(target_folder, ext.upper())))
    
    # Agrupar por nombre (sin importar mayúsculas/minúsculas)
    image_groups = {}
    for img_path in all_images:
        name = os.path.basename(img_path).lower()
        if name not in image_groups:
            image_groups[name] = []
        image_groups[name].append(img_path)
    
    # Eliminar duplicados (mantener solo el primero)
    removed_count = 0
    for name, paths in image_groups.items():
        if len(paths) > 1:
            print(f"📁 {name}: {len(paths)} copias encontradas")
            # Mantener el primero, eliminar el resto
            for path in paths[1:]:
                try:
                    os.remove(path)
                    print(f"  🗑️ Eliminado: {os.path.basename(path)}")
                    removed_count += 1
                except Exception as e:
                    print(f"  ❌ Error eliminando {path}: {e}")
    
    print(f"✅ Limpieza completada. {removed_count} archivos duplicados eliminados")

# Ejecutar limpieza
clean_duplicate_images_in_folder()

# Mostrar imágenes finales en la carpeta organizada (sin duplicados)
images = []
seen_names = set()
target_folder = "fotos_para_clasificar"

if os.path.exists(target_folder):
    for ext in ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.gif']:
        for img in glob.glob(os.path.join(target_folder, ext)):
            name = os.path.basename(img).lower()
            if name not in seen_names:
                images.append(img)
                seen_names.add(name)
    
    print(f"\n📸 Imágenes finales en {target_folder}/ ({len(images)}) - SIN DUPLICADOS:")
    for i, img in enumerate(sorted(images), 1):
        print(f"  {i}. {os.path.basename(img)}")
else:
    print(f"\n❌ No se encontró la carpeta {target_folder}/")


🧹 Limpiando imágenes duplicadas en fotos_para_clasificar/...
📁 botella plastico azul.jpg: 2 copias encontradas
  🗑️ Eliminado: botella plastico azul.jpg
📁 botella vidrio.jpg: 2 copias encontradas
  🗑️ Eliminado: botella vidrio.jpg
📁 brocoli.jpg: 2 copias encontradas
  🗑️ Eliminado: brocoli.jpg
📁 camisa.jpg: 2 copias encontradas
  🗑️ Eliminado: camisa.jpg
📁 carro viejo.jpg: 2 copias encontradas
  🗑️ Eliminado: carro viejo.jpg
📁 imagen de whatsapp 2025-10-21 a las 08.55.14_be760169.jpg: 2 copias encontradas
  🗑️ Eliminado: Imagen de WhatsApp 2025-10-21 a las 08.55.14_be760169.jpg
📁 lata abierta.jpg: 2 copias encontradas
  🗑️ Eliminado: lata abierta.jpg
📁 muñeca.jpg: 2 copias encontradas
  🗑️ Eliminado: muñeca.jpg
📁 basura-envase-plastico.jpeg: 2 copias encontradas
  🗑️ Eliminado: basura-envase-plastico.jpeg
📁 basura-vaso-plastico.jpeg: 2 copias encontradas
  🗑️ Eliminado: basura-vaso-plastico.jpeg
📁 basura1-metal.jpeg: 2 copias encontradas
  🗑️ Eliminado: basura1-metal.jpeg
📁 botella ver

In [31]:
# 🎯 CLASIFICADOR DE RESIDUOS - VERSIÓN MEJORADA V2
import glob
import os
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import matplotlib.pyplot as plt
from PIL import Image
import warnings
from pathlib import Path
import time

warnings.filterwarnings('ignore')

class ClasificadorResiduos:
    """Clasificador de residuos con interfaz mejorada y gestión de estado"""
    
    def __init__(self, target_folder="fotos_para_clasificar", contenedores_folder="contenedores"):
        self.target_folder = target_folder
        self.contenedores_folder = contenedores_folder
        self.learn_loaded = None
        
        # Configuración de contenedores y traducciones
        self.contenedor_mapping = {
            'cardboard': {'archivo': 'azul.png', 'color': 'Azul', 'emoji': '🔵', 'hex': '#2196F3'},
            'paper': {'archivo': 'azul.png', 'color': 'Azul', 'emoji': '🔵', 'hex': '#2196F3'},
            'glass': {'archivo': 'verde.png', 'color': 'Verde', 'emoji': '🟢', 'hex': '#4CAF50'},
            'plastic': {'archivo': 'amarillo.png', 'color': 'Amarillo', 'emoji': '🟡', 'hex': '#FFC107'},
            'metal': {'archivo': 'amarillo.png', 'color': 'Amarillo', 'emoji': '🟡', 'hex': '#FFC107'},
            'compost': {'archivo': 'marron.png', 'color': 'Marrón', 'emoji': '🟤', 'hex': '#795548'},
            'trash': {'archivo': 'marron.png', 'color': 'Marrón', 'emoji': '🟤', 'hex': '#795548'}
        }
        
        self.interpretaciones = {
            'cardboard': '📦 Cartón',
            'compost': '🌱 Compost/Orgánico',
            'glass': '🍷 Vidrio',
            'metal': '🔩 Metal',
            'paper': '📄 Papel',
            'plastic': '🥤 Plástico',
            'trash': '🗑️ Basura General'
        }
        
        self.info_contenedores = {
            'azul.png': ('Papel y Cartón', 'Periódicos, revistas, cajas de cartón, sobres'),
            'verde.png': ('Vidrio', 'Botellas, frascos, envases de vidrio'),
            'amarillo.png': ('Plástico y Metal', 'Botellas PET, latas, envases de plástico'),
            'marron.png': ('Orgánico', 'Restos de comida, cáscaras, residuos biodegradables')
        }
        
        # Historial de clasificaciones
        self.historial = []
        
    def validar_carpetas(self):
        """Valida que existan las carpetas necesarias"""
        if not os.path.exists(self.target_folder):
            return False, f"❌ No se encontró la carpeta {self.target_folder}/"
        
        if not os.path.exists(self.contenedores_folder):
            return False, f"❌ No se encontró la carpeta {self.contenedores_folder}/"
        
        return True, "✅ Carpetas validadas"
    
    def cargar_imagenes(self):
        """Carga todas las imágenes disponibles sin duplicados"""
        image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.gif']
        images = []
        seen_names = set()
        
        for ext in image_extensions:
            pattern = os.path.join(self.target_folder, ext)
            for img in glob.glob(pattern):
                name = os.path.basename(img).lower()
                if name not in seen_names:
                    images.append(img)
                    seen_names.add(name)
        
        return sorted(images)
    
    def clasificar_imagen(self, ruta_imagen):
        """Clasifica una imagen y retorna los resultados"""
        try:
            prediction = learn_loaded.predict(ruta_imagen)
            num = prediction[1].numpy().tolist()
            confidence = prediction[2].numpy()[num]
            clase = prediction[0]
            
            return {
                'clase': clase,
                'confianza': float(confidence),
                'interpretacion': self.interpretaciones.get(clase, clase),
                'error': None
            }
        except Exception as e:
            return {
                'clase': None,
                'confianza': None,
                'interpretacion': None,
                'error': str(e)
            }
    
    def mostrar_resultado(self, ruta_imagen, resultado):
        """Muestra el resultado de la clasificación con visualización mejorada"""
        if resultado['error']:
            print(f"❌ Error al clasificar: {resultado['error']}")
            return
        
        clase = resultado['clase']
        confianza = resultado['confianza']
        interpretacion = resultado['interpretacion']
        
        # Obtener información del contenedor
        contenedor_info = self.contenedor_mapping.get(clase)
        if not contenedor_info:
            print(f"❌ No hay contenedor definido para: {clase}")
            return
        
        contenedor_path = os.path.join(self.contenedores_folder, contenedor_info['archivo'])
        if not os.path.exists(contenedor_path):
            print(f"❌ No se encontró el archivo: {contenedor_info['archivo']}")
            return
        
        # Crear visualización mejorada
        try:
            fig = plt.figure(figsize=(16, 7), facecolor='white')
            gs = fig.add_gridspec(1, 3, width_ratios=[1, 1, 0.8], wspace=0.15)
            
            # Imagen del residuo (izquierda)
            ax1 = fig.add_subplot(gs[0, 0])
            img_residuo = Image.open(ruta_imagen)
            ax1.imshow(img_residuo)
            ax1.set_title("📸 Residuo Clasificado", fontsize=15, fontweight='bold', pad=15, 
                         color='#333')
            ax1.axis('off')
            
            # Imagen del contenedor (centro)
            ax2 = fig.add_subplot(gs[0, 1])
            img_contenedor = Image.open(contenedor_path)
            ax2.imshow(img_contenedor)
            titulo_contenedor = f"{contenedor_info['emoji']} Contenedor {contenedor_info['color']}"
            ax2.set_title(titulo_contenedor, fontsize=15, fontweight='bold', pad=15,
                         color=contenedor_info['hex'])
            ax2.axis('off')
            
            # Panel de información estilizado (derecha)
            ax3 = fig.add_subplot(gs[0, 2])
            ax3.axis('off')
            ax3.set_xlim(0, 1)
            ax3.set_ylim(0, 1)
            
            # Obtener información
            nombre_archivo = os.path.basename(ruta_imagen)
            tipo_residuo, ejemplos = self.info_contenedores.get(contenedor_info['archivo'], ('', ''))
            color_hex = contenedor_info['hex']
            
            # Crear cuadros de información con estilo
            y_pos = 0.95
            
            # Título
            ax3.add_patch(plt.Rectangle((0, y_pos - 0.12), 1, 0.12, 
                                       facecolor=color_hex, alpha=0.9, transform=ax3.transAxes))
            ax3.text(0.5, y_pos - 0.06, "📊 RESULTADO", 
                    ha='center', va='center', fontsize=14, fontweight='bold', 
                    color='white', transform=ax3.transAxes)
            
            y_pos -= 0.18
            
            # Archivo
            ax3.text(0.05, y_pos, "📁 Archivo:", fontsize=10, fontweight='bold', 
                    color='#555', transform=ax3.transAxes)
            ax3.text(0.05, y_pos - 0.06, nombre_archivo[:25] + '...' if len(nombre_archivo) > 25 else nombre_archivo, 
                    fontsize=9, color='#777', transform=ax3.transAxes, style='italic')
            
            y_pos -= 0.14
            
            # Categoría
            ax3.text(0.05, y_pos, "🏷️ Categoría:", fontsize=10, fontweight='bold', 
                    color='#555', transform=ax3.transAxes)
            ax3.text(0.05, y_pos - 0.06, interpretacion, 
                    fontsize=10, color=color_hex, fontweight='bold', transform=ax3.transAxes)
            
            y_pos -= 0.14
            
            # Confianza con barra
            ax3.text(0.05, y_pos, "📈 Confianza:", fontsize=10, fontweight='bold', 
                    color='#555', transform=ax3.transAxes)
            
            # Barra de confianza
            bar_y = y_pos - 0.08
            ax3.add_patch(plt.Rectangle((0.05, bar_y), 0.9, 0.03, 
                                       facecolor='#e0e0e0', transform=ax3.transAxes))
            ax3.add_patch(plt.Rectangle((0.05, bar_y), 0.9 * confianza, 0.03, 
                                       facecolor=color_hex, transform=ax3.transAxes))
            ax3.text(0.5, bar_y + 0.015, f"{confianza:.1%}", 
                    ha='center', va='center', fontsize=9, fontweight='bold',
                    color='white' if confianza > 0.5 else color_hex, transform=ax3.transAxes)
            
            y_pos -= 0.18
            
            # Contenedor
            ax3.add_patch(plt.Rectangle((0, y_pos - 0.01), 1, 0.18, 
                                       facecolor=color_hex, alpha=0.1, transform=ax3.transAxes))
            ax3.text(0.5, y_pos + 0.08, f"{contenedor_info['emoji']} CONTENEDOR {contenedor_info['color'].upper()}", 
                    ha='center', fontsize=11, fontweight='bold', 
                    color=color_hex, transform=ax3.transAxes)
            ax3.text(0.5, y_pos + 0.03, tipo_residuo, 
                    ha='center', fontsize=9, color='#666', transform=ax3.transAxes)
            
            y_pos -= 0.22
            
            # Ejemplos
            ax3.text(0.05, y_pos, "💡 Ejemplos:", fontsize=9, fontweight='bold', 
                    color='#555', transform=ax3.transAxes)
            
            # Dividir ejemplos en líneas
            palabras = ejemplos.split(', ')
            linea_actual = ""
            y_ejemplo = y_pos - 0.05
            
            for palabra in palabras:
                if len(linea_actual + palabra) < 25:
                    linea_actual += palabra + ", "
                else:
                    ax3.text(0.05, y_ejemplo, linea_actual.rstrip(', '), 
                            fontsize=8, color='#888', transform=ax3.transAxes)
                    y_ejemplo -= 0.04
                    linea_actual = palabra + ", "
            
            if linea_actual:
                ax3.text(0.05, y_ejemplo, linea_actual.rstrip(', '), 
                        fontsize=8, color='#888', transform=ax3.transAxes)
            
            plt.tight_layout()
            plt.show()
            
            # Agregar al historial
            self.historial.append({
                'imagen': nombre_archivo,
                'clase': interpretacion,
                'confianza': confianza,
                'contenedor': contenedor_info['color']
            })
            
        except Exception as e:
            print(f"❌ Error al mostrar resultado: {e}")
            import traceback
            traceback.print_exc()
    
    def clasificar_todas(self, imagenes, output_widget, progress_widget, contador_widget):
        """Clasifica todas las imágenes automáticamente"""
        total = len(imagenes)
        resultados_resumen = []
        
        with output_widget:
            clear_output(wait=True)
            print("🚀 Iniciando clasificación masiva...\n")
            
            for i, img_path in enumerate(imagenes, 1):
                # Actualizar progreso
                progreso = (i / total) * 100
                progress_widget.value = progreso
                progress_widget.description = f'{i}/{total}'
                
                # Clasificar
                resultado = self.clasificar_imagen(img_path)
                
                if not resultado['error']:
                    nombre = os.path.basename(img_path)
                    contenedor_info = self.contenedor_mapping.get(resultado['clase'])
                    
                    # Guardar resumen
                    resultados_resumen.append({
                        'nombre': nombre,
                        'clase': resultado['interpretacion'],
                        'confianza': resultado['confianza'],
                        'contenedor': contenedor_info['color'],
                        'emoji': contenedor_info['emoji']
                    })
                    
                    # Agregar al historial
                    self.historial.append({
                        'imagen': nombre,
                        'clase': resultado['interpretacion'],
                        'confianza': resultado['confianza'],
                        'contenedor': contenedor_info['color']
                    })
                    
                    print(f"✅ {i}/{total} - {nombre[:30]:<30} → {resultado['interpretacion']:<20} ({resultado['confianza']:.1%})")
                else:
                    print(f"❌ {i}/{total} - Error: {os.path.basename(img_path)}")
                
                time.sleep(0.05)  # Pequeña pausa para visualizar progreso
            
            # Mostrar resumen final
            print("\n" + "="*80)
            print("📊 RESUMEN DE CLASIFICACIÓN MASIVA")
            print("="*80)
            
            # Agrupar por contenedor
            por_contenedor = {}
            for r in resultados_resumen:
                cont = r['contenedor']
                if cont not in por_contenedor:
                    por_contenedor[cont] = []
                por_contenedor[cont].append(r)
            
            for contenedor, items in sorted(por_contenedor.items()):
                emoji = items[0]['emoji']
                print(f"\n{emoji} {contenedor.upper()} ({len(items)} imágenes):")
                for item in items:
                    print(f"  • {item['nombre'][:35]:<35} - {item['clase']:<20} {item['confianza']:.1%}")
            
            print("\n" + "="*80)
            print(f"✅ Clasificación completada: {len(resultados_resumen)}/{total} imágenes procesadas")
            print("="*80)
            
            # Actualizar contador
            contador_widget.value = (f"<div style='text-align: center; padding: 10px; font-size: 14px;'>"
                                    f"📊 Total de imágenes: <b>{total}</b> | "
                                    f"Clasificadas: <b>{len(self.historial)}</b></div>")
    
    def mostrar_historial(self):
        """Muestra el historial de clasificaciones"""
        if not self.historial:
            print("📋 No hay clasificaciones en el historial")
            return
        
        print("\n" + "="*70)
        print("📋 HISTORIAL DE CLASIFICACIONES")
        print("="*70)
        for i, item in enumerate(self.historial, 1):
            print(f"{i}. {item['imagen']}")
            print(f"   └─ {item['clase']} | Confianza: {item['confianza']:.1%} | Contenedor: {item['contenedor']}")
        print("="*70 + "\n")
    
    def crear_interfaz(self):
        """Crea y muestra la interfaz gráfica"""
        # Validar carpetas
        valido, mensaje = self.validar_carpetas()
        if not valido:
            print(mensaje)
            return
        
        # Cargar imágenes
        imagenes = self.cargar_imagenes()
        if not imagenes:
            print(f"❌ No hay imágenes en {self.target_folder}/")
            return
        
        # Crear widgets
        style_titulo = """
        <div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                    padding: 20px; border-radius: 10px; text-align: center;
                    box-shadow: 0 4px 6px rgba(0,0,0,0.1);'>
            <h1 style='color: white; margin: 0; font-size: 28px;'>
                🌍 Clasificador Inteligente de Residuos
            </h1>
            <p style='color: #e0e0e0; margin: 10px 0 0 0; font-size: 14px;'>
                Ayudando al planeta, un residuo a la vez
            </p>
        </div>
        """
        
        titulo_widget = widgets.HTML(value=style_titulo)
        
        # Selector de imagen
        imagen_dropdown = widgets.Dropdown(
            options=[(os.path.basename(img), img) for img in imagenes],
            description='📸 Imagen:',
            style={'description_width': '100px'},
            layout=widgets.Layout(width='400px', height='40px')
        )
        
        # Botones de acción
        clasificar_btn = widgets.Button(
            description='🎯 Clasificar',
            button_style='success',
            icon='check',
            layout=widgets.Layout(width='140px', height='40px')
        )
        
        clasificar_todas_btn = widgets.Button(
            description='🚀 Todas',
            button_style='primary',
            icon='bolt',
            layout=widgets.Layout(width='140px', height='40px'),
            tooltip='Clasificar todas las imágenes'
        )
        
        historial_btn = widgets.Button(
            description='📋 Historial',
            button_style='info',
            icon='list',
            layout=widgets.Layout(width='140px', height='40px')
        )
        
        limpiar_btn = widgets.Button(
            description='🗑️ Limpiar',
            button_style='warning',
            icon='trash',
            layout=widgets.Layout(width='140px', height='40px')
        )
        
        # Barra de progreso
        progress_bar = widgets.IntProgress(
            value=0,
            min=0,
            max=100,
            description='Progreso:',
            bar_style='success',
            style={'bar_color': '#667eea', 'description_width': '80px'},
            layout=widgets.Layout(width='600px', visibility='hidden')
        )
        
        # Contador de imágenes
        contador = widgets.HTML(
            value=f"<div style='text-align: center; padding: 10px; font-size: 14px;'>"
                  f"📊 Total de imágenes: <b>{len(imagenes)}</b> | "
                  f"Clasificadas: <b>{len(self.historial)}</b></div>"
        )
        
        # Output
        resultado_output = widgets.Output(layout=widgets.Layout(width='100%', margin='20px 0'))
        
        # Funciones de eventos
        def on_clasificar(b):
            with resultado_output:
                clear_output(wait=True)
                progress_bar.layout.visibility = 'hidden'
                if imagen_dropdown.value:
                    resultado = self.clasificar_imagen(imagen_dropdown.value)
                    self.mostrar_resultado(imagen_dropdown.value, resultado)
                    contador.value = (f"<div style='text-align: center; padding: 10px; font-size: 14px;'>"
                                    f"📊 Total de imágenes: <b>{len(imagenes)}</b> | "
                                    f"Clasificadas: <b>{len(self.historial)}</b></div>")
        
        def on_clasificar_todas(b):
            progress_bar.layout.visibility = 'visible'
            progress_bar.value = 0
            self.clasificar_todas(imagenes, resultado_output, progress_bar, contador)
        
        def on_historial(b):
            with resultado_output:
                clear_output(wait=True)
                progress_bar.layout.visibility = 'hidden'
                self.mostrar_historial()
        
        def on_limpiar(b):
            with resultado_output:
                clear_output(wait=True)
                progress_bar.layout.visibility = 'hidden'
                print("✨ Pantalla limpiada")
        
        clasificar_btn.on_click(on_clasificar)
        clasificar_todas_btn.on_click(on_clasificar_todas)
        historial_btn.on_click(on_historial)
        limpiar_btn.on_click(on_limpiar)
        
        # Layout
        controles = widgets.HBox(
            [imagen_dropdown, clasificar_btn, clasificar_todas_btn, historial_btn, limpiar_btn],
            layout=widgets.Layout(justify_content='center', margin='20px 0')
        )
        
        progress_container = widgets.HBox(
            [progress_bar],
            layout=widgets.Layout(justify_content='center', margin='10px 0')
        )
        
        # Instrucciones
        instrucciones = widgets.HTML(
            value="""
            <div style='background-color: #f8f9fa; padding: 15px; border-radius: 8px; 
                        border-left: 4px solid #667eea; margin: 20px 0;'>
                <h3 style='margin-top: 0; color: #667eea;'>📋 Instrucciones</h3>
                <ol style='margin: 0; padding-left: 20px;'>
                    <li><b>🎯 Clasificar:</b> Analiza una sola imagen seleccionada</li>
                    <li><b>🚀 Todas:</b> Clasifica automáticamente todas las imágenes</li>
                    <li><b>📋 Historial:</b> Ver todas las clasificaciones realizadas</li>
                    <li><b>🗑️ Limpiar:</b> Limpia la pantalla de resultados</li>
                </ol>
            </div>
            """
        )
        
        # Mostrar interfaz
        display(titulo_widget)
        display(contador)
        display(controles)
        display(progress_container)
        display(instrucciones)
        display(resultado_output)

# Función principal para ejecutar
def iniciar_clasificador():
    """Inicializa y ejecuta el clasificador"""
    clasificador = ClasificadorResiduos()
    clasificador.crear_interfaz()

# Ejecutar el clasificador
iniciar_clasificador()

HTML(value="\n        <div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);\n            …

HTML(value="<div style='text-align: center; padding: 10px; font-size: 14px;'>📊 Total de imágenes: <b>32</b> | …

HTML(value="\n            <div style='background-color: #f8f9fa; padding: 15px; border-radius: 8px; \n        …

Output(layout=Layout(margin='20px 0', width='100%'))